In [1]:
os.environ['SAGE_NUM_THREADS'] = '6'
nthreads = 6
pari.allocatemem(8589934592)

var('q t')

def vector_sum(u, v):
    return dict((x, u.get(x, 0) + v.get(x, 0)) for x in set(u)|set(v))

def vector_simpl(u):
    v = {}
    S = set()
    for a in set(u):
        b = sorted(a)
        b.reverse()
        b = tuple(b)
        S.add(b)
        v = vector_sum(v, {b:u[a]})
    return {a:v[a] for a in S if v[a] != 0}

def remove_zeros(u):
    return {a:u[a] for a in set(u) if u[a] != 0}

def univ_vector_simpl1(a):
    for i in range(len(a)-1):
        if a[i]<a[i+1]:
            return {a[0:i]+(a[i]+a[i+1],)+a[i+2:len(a)]:a[i+1]-a[i], a[0:i]+(a[i+1],)+(a[i],)+a[i+2:len(a)]:1}
    return {a:1}

def univ_vector_simpl(u):
    v={}
    for a in set(u):
        w = univ_vector_simpl1(a)
        v = vector_sum(v, {b:w[b]*u[a] for b in set(w)})
    if v == u:
        return v
    return univ_vector_simpl(v)

def scalar_times_vector(u, v):
    w = {}
    for a in set(u):
        for b in set(v):
            w = vector_sum(w, {a+b:u[a]*v[b]})
    return vector_simpl(w)

def several_virasoro(a, u):
    return univ_vector_simpl({a+b:u[b] for b in set(u)})

def Partitions1(n):
    if n < 0:
        return [[]]
    a=list(Partitions(n))
    a.reverse()
    return a

def Partitions2(n):
    if n == 0:
        return [[]]
    a = []
    b = Partitions1(n)
    for i in range(len(b)):
        if b[i][len(b[i])-1] > 1:
            a = a + [b[i]]
    return a

def number_greater2(a):
    k = 0
    if a == ():
        return 0
    for i in range(len(a)):
        if a[i]>1:
            k = k + 1
        else:
            return k
    return k

def number_of_ones(a):
    k = 0
    for i in range(len(a)):
        if a[i]==1:
            k = k+1
    return k

def Partitions1_into_parts(n):
    a = Partitions1(n)
    b = [[]]*(n+1)
    for i in range(len(a)):
        b[len(a[i])]= b[len(a[i])]+[i]
    return b

def simplify_vector(u):
    for i in range(len(u)):
        if u[i] != 0:
            return u/u[i]
        
def leading_matrix(A):
    A = A.echelon_form()
    a = []
    i = 0
    for j in range(A.ncols()):
        if i < A.nrows():
            if A[i,j] != 0:
                a = a + [j]
                i += 1      
    return a

def remove_zero_rows(A):
    for i in range(A.nrows()):
        if tuple(A[i]) == (0,)*(A.ncols()):
            A = A.delete_rows(range(i,A.nrows()))
            return A
    return A

def intersection(U, V):
    m = U.ncols()
    W = matrix(pari.mattranspose(pari.matintersect(pari(U.transpose()), pari(V.transpose()))))
    if W.nrows()==0:
        return matrix(0, m)
    return W

def insert_row(A, k, a):
    return matrix(A.rows()[:k] + [a] + A.rows()[k:])

def base_subspace(n):
    m1 = {(1,1):3, (2,):-4}
    m2 = {(1,1,1):5, (2,1):-12}
    a = Partitions1(n)
    b = Partitions1(n-2)
    c = Partitions1(n-3)
    bm1 = [several_virasoro(tuple(b[i]), m1) for i in range(len(b))]
    cm2 = [several_virasoro(tuple(c[i]), m2) for i in range(len(c))]
    A = matrix({(i,j): bm1[i].get(tuple(a[j]), 0) for j in range(len(a)) for i in range(len(b))}, sparse = False)
    B = matrix({(i,j): cm2[i].get(tuple(a[j]), 0) for j in range(len(a)) for i in range(len(c))}, sparse = False)
    A = A.stack(B)
    A = A.rref()
    A = remove_zero_rows(A)
    return A

def parts_up_to(n):
    return [Partitions1(i) for i in range(n+1)]

def parts2_up_to(n):
    return [Partitions2(i) for i in range(n+1)]

def expansion_t_q(n):
    a = Partitions1(n)
    b = Partitions1_into_parts(n)
    A = base_subspace(n)
    filter_list = list(leading_matrix(A))
    m = range(len(a))
    c = [x for x in m if x not in filter_list]
    B = matrix(0, len(c))
    d = [-1]*len(a)
    for i in range(len(filter_list)):
        d[filter_list[i]]=i
    k = 0
    for i in range(len(b)):
        for j in range(len(b[i])):
            if d[b[i][j]]!=-1:
                row = [A[d[b[i][j]], c[s]] for s in range(len(c))]
                B = insert_row(B, B.nrows(), row)
            else:
                row = [0]*len(c)
                for r in range(len(c)):
                    if c[r]==b[i][j]:
                        row[r]=1
                B = insert_row(B, B.nrows(), row)         
        B = B.echelon_form()
        B = remove_zero_rows(B)
        print((i,B.nrows()-k))
        k = B.nrows()

def prt2(a):
    st = ""
    if(a[len(a)-1]>1):
        for i in range(len(a)):
            if(i!= len(a)-1):
                st = st + "x(" + str(a[i]) + ")*"
            else:
                st = st + "x(" + str(a[i]) + ")"
        st = st + "*gen(1)"
    else:
        for i in range(len(a)-1):
            if(i!= len(a)-1):
                st = st + "x(" + str(a[i]) + ")*"
            else:
                st = st + "x(" + str(a[i]) + ")"
        st = st + "gen(2)"
    return st

def prt1(u):
    st = ""
    for a in set(u):
        st = st + str(u[a]) + "*" + prt2(a) + "+"
    st = st[:-1]
    return st

def prt(w):
    st = ""
    for i in range(len(w)):
        st = st + prt1(w[i]) + ","
    st = st[:-1]
    return st

def two_ones_equals_zero(u):
    return {a:u[a] for a in set(u) if number_of_ones(a) < 2}

@parallel
def inters_m_i(a, n, i):
    B = matrix(0, len(a))
    b = []
    w = []
    for k in range(r, i+1, nthreads):
        b = Partitions1_into_parts(m)[k]
        B2 = matrix(len(b), len(a))
        for j in range(len(b)):
            B2[j,b[j]] = 1
        B = B.stack(B2)
    C = intersection(B,A)
    w.extend([remove_zeros({tuple(a[b[t]]):C[j,b[t]] for t in range(len(b))}) for j in range(C.nrows())])
    return w

@parallel
def paral_inters(aa, n, r):
    c = [(m,i) for m in range(n+1) for i in range(m+1)]
    w = []
    for k in range(r, len(c), nthreads):
        m = c[k][0]
        i = c[k][1]
        a = Partitions1(m)
        A = aa[m]
        B = matrix(0, len(a))
        b = []
        for j in range(i+1):
            b = Partitions1_into_parts(m)[j]
            B2 = matrix(len(b), len(a))
            for r in range(len(b)):
                B2[r,b[r]] = 1
            B = B.stack(B2)
        C = intersection(B,A)
        w.extend([remove_zeros({tuple(a[b[t]]):C[j,b[t]] for t in range(len(b))}) for j in range(C.nrows())])
    return w

def paral_subspace_matrix(n):
    w = []
    aa = [base_subspace(m) for m in range(n+1)]
    args = []
    c = [(aa, n, r) for r in range(nthreads)]
    for r in range(nthreads):
        args += [(aa, n, r)]
    for output in paral_inters(args):
        w += output[1]
    ww = [two_ones_equals_zero(u) for u in w]
    www = [u for u in ww if u != {}]
    return www
    
def subspace_matrix(n): 
    w = []
    q = parts_up_to(n)
    for m in range(n+1):
        a = q[m]
        A = base_subspace(m)
        B = matrix(0, len(a))
        for i in range(m+1):
            b = Partitions1_into_parts(m)[i]
            B2 = matrix(len(b), len(a))
            for j in range(len(b)):
                B2[j, b[j]] = 1
            B = B.stack(B2)
            C = intersection(B, A)
            w.extend([remove_zeros({tuple(a[b[t]]):C[j,b[t]] for t in range(len(b))}) for j in range(C.nrows())])
    ww = [two_ones_equals_zero(u) for u in w]
    www = [u for u in ww if u != {}]
    return www

def to_partition(a):
    b = []
    max = 0
    while(a[max] == 0):
        max = max + 1
    i = len(a)-2
    while(i!=max-1):
        b = [len(a)-i]*a[i]+b
        i = i-1
    if a[len(a)-1]==2:
        b = b + [1]
    return b

def to_polynomial(a,n):
    b = [0]*n
    b[n-1] = number_of_ones(a)
    for i in range(len(a)):
        if a[i]!= 1:
            b[a[i]-2] = b[a[i]-2]+1
    return b        

def avoid_up_to(n):
    w = subspace_matrix(n)
    st = 'option(noredefine);'
    st = st + 'ring R = QQ, (x(2..' + str(n) + ')), (dp, C);'
    st = st + 'module M =' + prt(w) + ';'
    st = st + 'module G = std(M);'
    st = st + 'string result = "";'
    st = st + 'for(int i = 1; i<=size(G); i++){result = result + "[" + string(leadexp(G[i])) + "],";}'
    st = st + 'result = result[1..size(result)-1];'
    st = st + 'result = "[" + result + "]";'
    st = st + 'result;'
    st = singular.eval(st)
    avoid = eval(st)
    for i in range(len(avoid)):
        avoid[i][n-1] = avoid[i][n-1]-1
    return avoid

def avoid_up_to_v0(n):
    w = subspace_matrix(n)
    st = 'option(noredefine);'
    st = st + 'ring R = QQ, (x(' + str(n) + '..2)), (dp, C);' 
    st = st + 'module M =' + prt(w) + ';'
    st = st + 'module G = std(M);'
    st = st + 'string result = "";'
    st = st + 'for(int i = 1; i<=size(G); i++){result = result + "[" + string(leadexp(G[i])) + "],";}'
    st = st + 'result = result[1..size(result)-1];'
    st = st + 'result = "[" + result + "]";'
    st = st + 'result;'
    st = singular.eval(st)
    b = eval(st)
    avoid = [[1,1]]
    for i in range(len(b)):
        avoid = avoid + [to_partition(b[i])]
    return avoid

def paral_avoid_up_to(n):
    w = paral_subspace_matrix(n)
    st = 'option(noredefine);'
    st = st + 'ring R = QQ, (x(2..' + str(n) + ')), (dp, C);'
    st = st + 'module M =' + prt(w) + ';'
    st = st + 'module G = std(M);'
    st = st + 'string result = "";'
    st = st + 'for(int i = 1; i<=size(G); i++){result = result + "[" + string(leadexp(G[i])) + "],";}'
    st = st + 'result = result[1..size(result)-1];'
    st = st + 'result = "[" + result + "]";'
    st = st + 'result;'
    st = singular.eval(st)
    avoid = eval(st)
    for i in range(len(avoid)):
        avoid[i][n-1] = avoid[i][n-1]-1
    return avoid

def paral_avoid_up_to_v0(n):
    w = paral_subspace_matrix(n)
    st = 'option(noredefine);'
    st = st + 'ring R = QQ, (x(' + str(n) + '..2)), (dp, C);' 
    st = st + 'module M =' + prt(w) + ';'
    st = st + 'module G = std(M);'
    st = st + 'string result = "";'
    st = st + 'for(int i = 1; i<=size(G); i++){result = result + "[" + string(leadexp(G[i])) + "],";}'
    st = st + 'result = result[1..size(result)-1];'
    st = st + 'result = "[" + result + "]";'
    st = st + 'result;'
    st = singular.eval(st)
    b = eval(st)
    avoid = [[1,1]]
    for i in range(len(b)):
        avoid = avoid + [to_partition(b[i])]
    return avoid

def p_divides_q(p,q,n):
    for i in range(n):
        if p[i]>q[i]:
            return false
    return true

def a_contains_any_of(a, avoid, n):
    p = to_polynomial(a, n)
    if p[n-1]>1:
        return true
    for i in range(len(avoid)):
        if p[n-1] == avoid[i][n-1] and p_divides_q(avoid[i][:-1], p[:-1], n-1):
            return true
    return false

def base_up_to(n):
    base = [[[]], [[1]], [[2]]]
    avoid = avoid_up_to(n)
    q = parts_up_to(n)
    for i in range(3, n+1):
        add = []
        a = q[i]
        for j in range(len(a)):
            if not a_contains_any_of(a[j], avoid, n):
                add = add + [a[j]]
        base = base + [add]
    return base

def paral_base_up_to(n):
    base = [[[]], [[1]], [[2]]]
    avoid = paral_avoid_up_to(n)
    q = parts_up_to(n)
    for i in range(3, n+1):
        add = []
        a = q[i]
        for j in range(len(a)):
            if not a_contains_any_of(a[j], avoid, n):
                add = add + [a[j]]
        base = base + [add]
    return base

def series_t_q(w):
    series = 0
    for i in range(len(w)):
        series_t = 0
        for j in range(len(w[i])):
            series_t = series_t + t^len(w[i][j])
        series = series + series_t*q^i
    return series

def last_equal_one(w):
    ww = [[]]
    for i in range(1, len(w)):
        a = []
        for j in range(len(w[i])):
            if w[i][j][len(w[i][j])-1]==1:
                a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def last_greater_one(w):
    ww = [[]]
    for i in range(1, len(w)):
        a = []
        for j in range(len(w[i])):
            if w[i][j][len(w[i][j])-1]>1:
                a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def last_equal_two(w):
    ww = [[]]
    for i in range(1, len(w)):
        a = []
        for j in range(len(w[i])):
            if w[i][j][len(w[i][j])-1]==2:
                a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def last_greater_two(w):
    ww = [[]]
    for i in range(1, len(w)):
        a = []
        for j in range(len(w[i])):
            if w[i][j][len(w[i][j])-1]>2:
                a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def last_equal_two_belast_equal_four(w):
    ww = [[]]
    for i in range(1, len(w)):
        a = []
        for j in range(len(w[i])):
            if w[i][j][len(w[i][j])-1]==2 and w[i][j][len(w[i][j])-2]==4:
                a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def last_equal_two_belast_greater_four(w):
    ww = [[]]
    for i in range(1, len(w)):
        a = []
        for j in range(len(w[i])):
            if w[i][j][len(w[i][j])-1]==2 and w[i][j][len(w[i][j])-2]>4:
                a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def last_equal_two_belast_equal_four_bebelast_equal_six(w):
    ww = [[]]
    for i in range(1, len(w)):
        a = []
        for j in range(len(w[i])):
            if w[i][j][len(w[i][j])-1]==2 and w[i][j][len(w[i][j])-2]==4 and w[i][j][len(w[i][j])-3]==6:
                a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def last_equal_two_belast_equal_four_bebelast_greater_six(w):
    ww = [[]]
    for i in range(1, len(w)):
        a = []
        for j in range(len(w[i])):
            if w[i][j][len(w[i][j])-1]==2 and w[i][j][len(w[i][j])-2]==4 and w[i][j][len(w[i][j])-3]>6:
                a = a + [w[i][j]]
        ww = ww + [a]
    return ww

PARI stack size set to 8589934592 bytes, maximum size set to 8589934592


In [2]:
%time base_up_to(17)

CPU times: user 10.4 s, sys: 171 ms, total: 10.6 s
Wall time: 10.7 s


[[[]],
 [[1]],
 [[2]],
 [[3]],
 [[3, 1], [4]],
 [[4, 1], [5]],
 [[4, 2], [5, 1], [6]],
 [[4, 3], [5, 2], [6, 1], [7]],
 [[4, 4], [5, 3], [6, 2], [7, 1], [8]],
 [[5, 3, 1], [5, 4], [6, 3], [7, 2], [8, 1], [9]],
 [[5, 4, 1], [5, 5], [6, 3, 1], [6, 4], [7, 3], [8, 2], [9, 1], [10]],
 [[5, 5, 1],
  [6, 4, 1],
  [6, 5],
  [7, 3, 1],
  [7, 4],
  [8, 3],
  [9, 2],
  [10, 1],
  [11]],
 [[5, 5, 2],
  [6, 4, 2],
  [6, 5, 1],
  [6, 6],
  [7, 4, 1],
  [7, 5],
  [8, 3, 1],
  [8, 4],
  [9, 3],
  [10, 2],
  [11, 1],
  [12]],
 [[6, 4, 3],
  [6, 5, 2],
  [6, 6, 1],
  [7, 4, 2],
  [7, 5, 1],
  [7, 6],
  [8, 4, 1],
  [8, 5],
  [9, 3, 1],
  [9, 4],
  [10, 3],
  [11, 2],
  [12, 1],
  [13]],
 [[6, 5, 3],
  [6, 6, 2],
  [7, 4, 3],
  [7, 5, 2],
  [7, 6, 1],
  [7, 7],
  [8, 4, 2],
  [8, 5, 1],
  [8, 6],
  [9, 4, 1],
  [9, 5],
  [10, 3, 1],
  [10, 4],
  [11, 3],
  [12, 2],
  [13, 1],
  [14]],
 [[6, 6, 3],
  [7, 4, 4],
  [7, 5, 3],
  [7, 6, 2],
  [7, 7, 1],
  [8, 4, 3],
  [8, 5, 2],
  [8, 6, 1],
  [8, 7],
  [9, 

In [3]:
%time paral_base_up_to(17)

CPU times: user 1.5 s, sys: 169 ms, total: 1.67 s
Wall time: 5.02 s


[[[]],
 [[1]],
 [[2]],
 [[3]],
 [[3, 1], [4]],
 [[4, 1], [5]],
 [[4, 2], [5, 1], [6]],
 [[4, 3], [5, 2], [6, 1], [7]],
 [[4, 4], [5, 3], [6, 2], [7, 1], [8]],
 [[5, 3, 1], [5, 4], [6, 3], [7, 2], [8, 1], [9]],
 [[5, 4, 1], [5, 5], [6, 3, 1], [6, 4], [7, 3], [8, 2], [9, 1], [10]],
 [[5, 5, 1],
  [6, 4, 1],
  [6, 5],
  [7, 3, 1],
  [7, 4],
  [8, 3],
  [9, 2],
  [10, 1],
  [11]],
 [[5, 5, 2],
  [6, 4, 2],
  [6, 5, 1],
  [6, 6],
  [7, 4, 1],
  [7, 5],
  [8, 3, 1],
  [8, 4],
  [9, 3],
  [10, 2],
  [11, 1],
  [12]],
 [[6, 4, 3],
  [6, 5, 2],
  [6, 6, 1],
  [7, 4, 2],
  [7, 5, 1],
  [7, 6],
  [8, 4, 1],
  [8, 5],
  [9, 3, 1],
  [9, 4],
  [10, 3],
  [11, 2],
  [12, 1],
  [13]],
 [[6, 5, 3],
  [6, 6, 2],
  [7, 4, 3],
  [7, 5, 2],
  [7, 6, 1],
  [7, 7],
  [8, 4, 2],
  [8, 5, 1],
  [8, 6],
  [9, 4, 1],
  [9, 5],
  [10, 3, 1],
  [10, 4],
  [11, 3],
  [12, 2],
  [13, 1],
  [14]],
 [[6, 6, 3],
  [7, 4, 4],
  [7, 5, 3],
  [7, 6, 2],
  [7, 7, 1],
  [8, 4, 3],
  [8, 5, 2],
  [8, 6, 1],
  [8, 7],
  [9, 